In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('../../'))

import numpy as np
import arrayfire as af

from dg_maxwell import params
from dg_maxwell import msh_parser
from dg_maxwell import advection_2d
from dg_maxwell import global_variables as gvar
from dg_maxwell import utils
from dg_maxwell import advection_2d_arbit_mesh as a2d_arbit_mesh
from dg_maxwell import wave_equation_2d

af.set_backend(params.backend)
af.set_device(params.device)

gmshtranslator friendly reminder: rules and actions prototypes


def node_condition(tag,x,y,z,physgroups): 
def node_action(tag,x,y,z):
def element_condition(eletag,eletype,physgrp,nodes):
def element_action(eletag,eletype,physgrp,nodes):


In [2]:
params.mesh_file = '../read_and_plot_mesh/mesh/square_10_10.msh'
advec_var = gvar.advection_variables(params.N_LGL, params.N_quad,
                                     params.x_nodes, params.N_Elements,
                                     params.c, params.total_time, params.wave,
                                     params.c_x, params.c_y, params.courant,
                                     params.mesh_file, params.total_time_2d)
new_elements = msh_parser.rearrange_element_edges(advec_var.elements, advec_var)
advec_var.reassign_2d_elements(new_elements)

/home/ubermensch/.local/anaconda3/lib/python3.6/site-packages/numpy/lib/polynomial.py:1193: FutureWarning: In the future extra properties will not be copied across when constructing one poly1d from another
  other = poly1d(other)
/home/ubermensch/.local/anaconda3/lib/python3.6/site-packages/numpy/lib/polynomial.py:1220: FutureWarning: In the future extra properties will not be copied across when constructing one poly1d from another
  other = poly1d(other)


gmshTranslator: Initializing...
gmshTranslator: Mesh has 441 nodes.
gmshTranslator: Mesh has 144 elements.
gmshTranslator: Processed 589 lines.
gmshTranslator: There are 1 physical groups available: 
gmshTranslator:      > 0
gmshTranslator: Parsing nodes
gmshTranslator: Parsing elements
gmshTranslator: No rules for elements... skipping elements.
gmshTranslator: Parsing nodes
gmshTranslator: No rules for nodes... skipping nodes.
gmshTranslator: Parsing elements
advection_variables __init__ completed


In [3]:
u = af.tile(advec_var.u_e_ij, d0 = 1, d1 = 1, d2 = 2)

In [102]:
# Volume integral term
## Common variables for both
shape_u_n = utils.shape(u)

dLp_xi_ij_Lq_eta_ij = advec_var.dLp_Lq
dLq_eta_ij_Lp_xi_ij = advec_var.dLq_Lp
dxi_dx   = 10.
deta_dy  = 10.
jacobian = 100.
c_x = params.c_x
c_y = params.c_y


In [49]:
# if (params.volume_integrand_scheme_2d == 'Lobatto' and params.N_LGL == params.N_quad):
w_i = af.flat(af.transpose(af.tile(advec_var.lobatto_weights_quadrature,
                                   1, params.N_LGL)))
w_j = af.tile(advec_var.lobatto_weights_quadrature,
              params.N_LGL)
wi_wj = w_i * w_j

wi_wj_dLp_xi = af.tile(af.broadcast(utils.multiply, wi_wj, advec_var.dLp_Lq),
                       d0 = 1, d1 = 1, d2 = 1, d3 = shape_u_n[2])

# print(wi_wj_dLp_xi.shape)
volume_integrand_ij_1_sp = c_x * dxi_dx \
                         * af.broadcast(utils.multiply,
                                        wi_wj_dLp_xi,
                                        af.reorder(u, d0 = 0, d1 = 1, d2 = 3, d3 = 2)) \
                                        / jacobian

wi_wj_dLq_eta = af.tile(af.broadcast(utils.multiply, w_i * w_j, advec_var.dLq_Lp),
                        d0 = 1, d1 = 1, d2 = 1, d3 = shape_u_n[2])

volume_integrand_ij_2_sp = c_y * deta_dy \
                         * af.broadcast(utils.multiply,
                                        wi_wj_dLq_eta,
                                        af.reorder(u, d0 = 0, d1 = 1, d2 = 3, d3 = 2)) \
                                        / jacobian
# print(af.sum(volume_integrand_ij_1_sp + volume_integrand_ij_2_sp, dim = 0).shape)
volume_integral = af.reorder(af.sum(volume_integrand_ij_1_sp + volume_integrand_ij_2_sp,
                                    dim = 0), d0 = 2, d1 = 1, d2 = 3, d3 = 0)
print(volume_integral.shape)

(64, 100, 2)


# Testing the new volume integral function (Part 1)

In [5]:
volume_integral_ref = a2d_arbit_mesh.volume_integral_old(u[:, :, 0], advec_var = advec_var)

In [116]:
print(af.all_true(af.abs(ref_volume_integral - volume_integral[:, :, 1]) < 1e-14))
# [It's working]

1.0


In [32]:
print(af.reorder(u, d0 = 0, d1 = 1, d2 = 3, d3 = 2).shape)

(64, 100, 1, 2)


In [110]:
volume_integrand_ij_1 = c_x * dxi_dx * af.broadcast(utils.multiply,
                                                    af.tile(dLp_xi_ij_Lq_eta_ij, d0 = 1, d1 = 1, d2 = 1, d3 = shape_u_n[2]),
                                                    af.reorder(u, d0 = 0, d1 = 1, d2 = 3, d3 = 2)) / jacobian

volume_integrand_ij_2 = c_y * deta_dy * af.broadcast(utils.multiply,
                                                     af.tile(dLq_eta_ij_Lp_xi_ij, d0 = 1, d1 = 1, d2 = 1, d3 = shape_u_n[2]),
                                                     af.reorder(u, d0 = 0, d1 = 1, d2 = 3, d3 = 2)) / jacobian

volume_integrand_ij = af.moddims(volume_integrand_ij_1 + volume_integrand_ij_2,
                                 d0 = params.N_LGL ** 2,
                                 d1 = (params.N_LGL ** 2) * 100,
                                 d2 = 1,
                                 d3 = shape_u_n[2])

# [FINE TILL HERE]

volume_integrand_ij = af.moddims(af.reorder(volume_integrand_ij, d0 = 0,
                                            d1 = 1, d2 = 3, d3 = 2),
                                 d0 = params.N_LGL * params.N_LGL,
                                 d1 = params.N_LGL * params.N_LGL * advec_var.elements.shape[0] * shape_u_n[2],
                                 d2 = 1)

print(volume_integrand_ij.shape)
lagrange_interpolation = wave_equation_2d.lag_interpolation_2d(volume_integrand_ij,
                                                               advec_var.Li_Lj_coeffs)
#                                     d0 = params.N_LGL, d1 = params.N_LGL,
#                                     d2 = params.N_LGL * params.N_LGL * advec_var.elements.shape[0],
#                                     d3 = shape_u_n[2])

# print(lagrange_interpolation.shape)
# [FINE TILL NOW]

print(lagrange_interpolation[:, :, :, 0].shape)

volume_integrand_total = utils.integrate_2d_multivar_poly(lagrange_interpolation,
                                                          params.N_quad,'gauss', advec_var)
print(volume_integrand_total.shape)
volume_integral = af.moddims(volume_integrand_total,
                             d0 = params.N_LGL * params.N_LGL * advec_var.elements.shape[0],
                             d1 = shape_u_n[2])
print(volume_integral[:, :, 0].shape)

volume_integral        = af.transpose(af.moddims(volume_integral,
                                                 d0 = advec_var.elements.shape[0],
                                                 d1 = params.N_LGL ** 2,
                                                 d2 = shape_u_n[2]))

# # print(volume_integral)


(64, 12800)
(8, 8, 12800)
(12800,)
(6400, 2)


# Formal implementation of the function

In [6]:
def volume_integral(u, advec_var):
    '''
    Vectorize, p, q, moddims.
    '''
    shape_u_n = utils.shape(u)
    dLp_xi_ij_Lq_eta_ij = advec_var.dLp_Lq
    dLq_eta_ij_Lp_xi_ij = advec_var.dLq_Lp
    dxi_dx   = 10.
    deta_dy  = 10.
    jacobian = 100.
    c_x = params.c_x
    c_y = params.c_y

    if (params.volume_integrand_scheme_2d == 'Lobatto' and params.N_LGL == params.N_quad):
        w_i = af.flat(af.transpose(af.tile(advec_var.lobatto_weights_quadrature,
                                           1, params.N_LGL)))
        w_j = af.tile(advec_var.lobatto_weights_quadrature,
                      params.N_LGL)
        wi_wj = w_i * w_j

        wi_wj_dLp_xi = af.tile(af.broadcast(utils.multiply, wi_wj,
                                            advec_var.dLp_Lq),
                               d0 = 1, d1 = 1, d2 = 1, d3 = shape_u_n[2])

        volume_integrand_ij_1_sp = c_x * dxi_dx \
                                 * af.broadcast(utils.multiply,
                                                wi_wj_dLp_xi,
                                                af.reorder(u, d0 = 0, d1 = 1,
                                                           d2 = 3, d3 = 2)) \
                                                / jacobian

        wi_wj_dLq_eta = af.tile(af.broadcast(utils.multiply,
                                             w_i * w_j,
                                             advec_var.dLq_Lp),
                                d0 = 1, d1 = 1, d2 = 1, d3 = shape_u_n[2])

        volume_integrand_ij_2_sp = c_y * deta_dy \
                                 * af.broadcast(utils.multiply,
                                                wi_wj_dLq_eta,
                                                af.reorder(u, d0 = 0, d1 = 1,
                                                           d2 = 3, d3 = 2)) \
                                                / jacobian

        volume_integral = af.reorder(af.sum(
            volume_integrand_ij_1_sp + volume_integrand_ij_2_sp, dim = 0), \
                d0 = 2, d1 = 1, d2 = 3, d3 = 0)

    else:
        volume_integrand_ij_1 = c_x * dxi_dx * af.broadcast(utils.multiply,
                                                            af.tile(dLp_xi_ij_Lq_eta_ij,
                                                                    d0 = 1, d1 = 1, d2 = 1,
                                                                    d3 = shape_u_n[2]),
                                                            af.reorder(u, d0 = 0, d1 = 1,
                                                                       d2 = 3, d3 = 2)) \
                                                            / jacobian

        volume_integrand_ij_2 = c_y * deta_dy * af.broadcast(utils.multiply,
                                                             af.tile(dLq_eta_ij_Lp_xi_ij,
                                                                     d0 = 1, d1 = 1, d2 = 1,
                                                                     d3 = shape_u_n[2]),
                                                             af.reorder(u, d0 = 0, d1 = 1,
                                                                        d2 = 3, d3 = 2)) \
                                                             / jacobian

        volume_integrand_ij = af.moddims(volume_integrand_ij_1 + volume_integrand_ij_2,
                                         d0 = params.N_LGL ** 2,
                                         d1 = (params.N_LGL ** 2) * 100,
                                         d2 = 1,
                                         d3 = shape_u_n[2])

        volume_integrand_ij = af.moddims(af.reorder(volume_integrand_ij, d0 = 0,
                                                    d1 = 1, d2 = 3, d3 = 2),
                                         d0 = params.N_LGL * params.N_LGL,
                                         d1 = params.N_LGL * params.N_LGL \
                                            * advec_var.elements.shape[0] \
                                            * shape_u_n[2],
                                         d2 = 1)

        lagrange_interpolation = wave_equation_2d.lag_interpolation_2d(volume_integrand_ij,
                                                                       advec_var.Li_Lj_coeffs)

        volume_integrand_total = utils.integrate_2d_multivar_poly(lagrange_interpolation,
                                                                  params.N_quad,'gauss',
                                                                  advec_var)

        volume_integral = af.moddims(volume_integrand_total,
                                     d0 = params.N_LGL \
                                        * params.N_LGL \
                                        * advec_var.elements.shape[0],
                                     d1 = shape_u_n[2])

        volume_integral = af.transpose(af.moddims(volume_integral,
                                                  d0 = advec_var.elements.shape[0],
                                                  d1 = params.N_LGL ** 2,
                                                  d2 = shape_u_n[2]))

    return volume_integral


In [6]:
volume_integral_test = a2d_arbit_mesh.volume_integral(u, advec_var)

In [7]:
af.all_true(af.abs(volume_integral_ref - volume_integral_test[:, :, 1]) < 1e-14)

1.0